In [ ]:
from pathlib import Path

In [ ]:
def process_line_rec(line):
    replacement_map = dict()
    replacement_map["core::DtypeUtil::ToString"] = ".ToString()"
    replacement_map["core::DtypeUtil::ByteSize"] = ".ByteSize()"
    replacement_map["DtypeUtil::ToString"] = ".ToString()"
    replacement_map["DtypeUtil::ByteSize"] = ".ByteSize()"
    for pattern, new_func in replacement_map.items():
        if pattern in line:
            print(f"{pattern} found in {line}")
            start_idx = line.find(pattern)
            start_idx += len(pattern)
            if line[start_idx] != "(":
                continue
            parentheses_count = 0
            end_idx = start_idx
            for i in range(start_idx, len(line), 1):
                if line[i] == "(":
                    parentheses_count += 1
                elif line[i] == ")":
                    parentheses_count -= 1
                if parentheses_count == 0:
                    end_idx = i
                    break
            if end_idx == start_idx:
                raise RuntimeError(f"Wrong end_idx: {line}")
            line = line[:start_idx - len(pattern)] + line[
                start_idx + 1:end_idx] + new_func + line[end_idx + 1:]
        
    return line
        
def process_line(line):
    new_line = process_line_rec(line)
    while new_line != line:
        line = new_line
        new_line = process_line_rec(line)
    return new_line

In [ ]:
paths = list(Path("/home/yixing/repo/Open3D/cpp").rglob("*.h"))
paths.extend(list(Path("/home/yixing/repo/Open3D/cpp").rglob("*.cpp")))
paths.extend(list(Path("/home/yixing/repo/Open3D/cpp").rglob("*.cu")))
for path in paths:
    with open(path, "r") as f:
        lines = f.readlines()
    lines = [process_line(line) for line in lines]
    with open(path, "w") as f:
        for line in lines:
            f.write(line)